In [1]:
import urllib3
from bs4 import BeautifulSoup
from selenium import webdriver
import re
import pandas as pd

In [2]:
## ToDo:
# 1) Complete - Any booked dates actually have a attribute called title that contains the cost.  
#    I need to pull this cost out and strip the $ from it and sum it per month for 
#    each house. This can give me a good estimate of revenue per house rather than
#    the vacancy per house which is more reflective of what I want.

In [3]:
# Some additional issues with the webdriver for firefox is that you have to download the geckodriver.
# This driver needs to be downloaded and then added to the system variable path.  However, it doesn't 
# seem like adding the path works so I had to move the geckodriver file to /usr/local/bin

# https://stackoverflow.com/questions/40208051/selenium-using-python-geckodriver-executable-needs-to-be-in-path

driver = webdriver.Firefox()

In [4]:
### below set of code was initially used for web request but I found out that 
### each individual home is dynamically generated so I needed selenium to pull
###this with a Firefox driver.
#springbrook='https://www.spring-brook.com/vacation-homes/all-vacation-homes/'
#http = urllib3.PoolManager()
#r = http.request('GET',springbrook)
#soup = BeautifulSoup(r.data)


driver.get('https://www.spring-brook.com/vacation-homes/all-vacation-homes/')

html = driver.page_source
soup = BeautifulSoup(html)

/home/onionknight/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/onionknight/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [5]:
urlList = []
regex = r"^(http|https)://"
#identify all html a tags and loop through them
all_links = soup.find_all("a")
for link in all_links:
    # for this specific website, individual housing url is located in ng-href witht he same url repeated in href.
    # since ng-href is more unique, we're using this to pull the data
    urlstr=link.get("ng-href")
    # check that it is a string and it contains http or https in the beginning, if so, add to list.
    if isinstance(urlstr, str) and re.search(regex,urlstr):
        urlList.append(urlstr)
# converting a list into a set will make each individual value unique and therefore getting rid of duplicate URLs
urlListnew=set(urlList)
urlListnew=list(urlListnew)

In [6]:
def loopHousing(urlList):
    fullArray=[["url","year","month","avail","booked","totCost","sleeps","bedrooms","bathrooms","pets"]]
    # Loop through each housing
    for url in urlList:
        #setup firefox driver to render page with dynamic html
        driver.get(url)
        indivHtml = driver.page_source
        yummysoup = BeautifulSoup(indivHtml)
        
        #pull metadata about the house
        metaDataHtml=yummysoup.find('div', class_="row property-information")
        metaDataListHtml = metaDataHtml.find_all('div', class_="col-md-3 col-sm-3 col-xs-3")
        for metaData in metaDataListHtml:
            if "Sleeps" in metaData.span.string:
                sleeps=metaData.strong.string
            if "Bedrooms" in metaData.span.string:
                bedrooms=metaData.strong.string
            if "Bathrooms" in metaData.span.string:
                bathrooms=metaData.strong.string
            if "Pets" in metaData.span.string:
                pets=metaData.strong.string
        metaDataList=[sleeps,bedrooms,bathrooms,pets]
        
        #availability data by month
        groupFirst=yummysoup.find('div', class_="ui-datepicker-group ui-datepicker-group-first")
        groupMiddle=yummysoup.find('div', class_="ui-datepicker-group ui-datepicker-group-middle")
        groupLast=yummysoup.find('div', class_="ui-datepicker-group ui-datepicker-group-last")
        fullArray.append(CtBookings(groupFirst,url)+metaDataList)
        fullArray.append(CtBookings(groupMiddle,url)+metaDataList)
        fullArray.append(CtBookings(groupLast,url)+metaDataList)    
    return(fullArray)


def CtBookings(group,url):
    # initialize count variables
    availCt=0
    bookedCt=0
    bookedCost=0
    # define month and year
    month=group.find('span', class_='ui-datepicker-month').string
    year=group.find('span', class_='ui-datepicker-year').string
    # useful to display that it is still working
    print(url+" "+month+" "+year)
    # dig through html structure
    calendarTable=group.find('table')
    calendarTbody=calendarTable.find('tbody')
    calendarTr=calendarTbody.find_all('tr')
    # loop through every row of every calendar
    for tr in calendarTr:
        calendarTd=tr.find_all('td')
        for td in calendarTd:
            classData='^'+' '.join(td['class']).strip().replace(" ", "^")+'^'
            # check for specific classes
            if '^available^' in classData:
                availCt+=1
            if '^booked^' in classData:
                bookedCost+=float(td['title'][1:])
                bookedCt+=1
            #print(td.string+' '+classData)
    mainList=[url,year,month,availCt,bookedCt,bookedCost]
    return(mainList)

#CtBookings(groupMiddle,urlListnew[1])


In [7]:
fullVal=loopHousing(urlListnew)
newVal=pd.DataFrame(fullVal)
newVal.to_csv('~/Documents/SpringBrookMetaData.csv')

/home/onionknight/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/onionknight/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


https://www.spring-brook.com/223608 January 2018
https://www.spring-brook.com/223608 February 2018
https://www.spring-brook.com/223608 March 2018
https://www.spring-brook.com/237300 January 2018
https://www.spring-brook.com/237300 February 2018
https://www.spring-brook.com/237300 March 2018
https://www.spring-brook.com/237341 January 2018
https://www.spring-brook.com/237341 February 2018
https://www.spring-brook.com/237341 March 2018
https://www.spring-brook.com/237296 January 2018
https://www.spring-brook.com/237296 February 2018
https://www.spring-brook.com/237296 March 2018
https://www.spring-brook.com/230218 January 2018
https://www.spring-brook.com/230218 February 2018
https://www.spring-brook.com/230218 March 2018
https://www.spring-brook.com/230463 January 2018
https://www.spring-brook.com/230463 February 2018
https://www.spring-brook.com/230463 March 2018
https://www.spring-brook.com/237298 January 2018
https://www.spring-brook.com/237298 February 2018
https://www.spring-brook.

https://www.spring-brook.com/231093 January 2018
https://www.spring-brook.com/231093 February 2018
https://www.spring-brook.com/231093 March 2018
https://www.spring-brook.com/230209 January 2018
https://www.spring-brook.com/230209 February 2018
https://www.spring-brook.com/230209 March 2018
https://www.spring-brook.com/278176 January 2018
https://www.spring-brook.com/278176 February 2018
https://www.spring-brook.com/278176 March 2018
https://www.spring-brook.com/223688 January 2018
https://www.spring-brook.com/223688 February 2018
https://www.spring-brook.com/223688 March 2018
https://www.spring-brook.com/231081 January 2018
https://www.spring-brook.com/231081 February 2018
https://www.spring-brook.com/231081 March 2018


In [18]:
x=1/6
import numpy as np
mzhary=np.array([x,1], dtype=float)
mzhary[0]
type(6)

int